In [1]:
from auxFuncs import *

In [2]:
print("...ok! Python3 loaded.")

...ok! Python3 loaded.


In [4]:
now = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
sourceDir = "/Users/Patrick Wilk/Documents/RL/JJ_first_Look/SURPL-V2-JJ_first_look/JJ_Jupyter/results/"
fileOut = open(sourceDir + "/PPO.txt", "w+")
tfLogs = sourceDir + "/logs"
now

'02_09_2022_15_21_46'

In [5]:
%load_ext tensorboard

In [6]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(sourceDir + "/logs", histogram_freq=1)

In [7]:
# ------------------------------
#       SMART BUILDING
# ------------------------------
class SmartBuildingEnv(Env):
    def __init__(self):
        self.viewer = None
        self.load = 0
        self.demand = []
        self.timeStep = 0
        self.deltaUtilization = 0
        self.penalty = 0
        self.action_space = None
        self.observation_space = None
        self.reset()
        
    def reset(self):
        # self.load = 0
        # MOVE TO 24 HOURS FOR INITAL RESULTS NOW THAT PRELIM RESULTS ESTABLISHED
        self.demand = [3, 1, 1]
        # self.demand = np.array([3,1,1])
        # self.demand = [3, 2, 1]
        # self.timeStep +=1 == self.demand[self.timeStep]
        self.timeStep = 0
        self.deltaUtilization = abs(self.demand[self.timeStep] - self.load)
        self.penalty = (self.deltaUtilization) ** 2
        self.action_space = Box(low=0, high=self.demand[self.timeStep], shape=(1,), dtype=float)
        # OBSERVATION SPACE CAN ALSO INCLUDE DEMAND CHARGE, TIME, ETC -> 1 INSUFFICIENT FOR MORE RESULTS
        self.observation_space = Box(low=0, high=self.deltaUtilization, shape=(1,), dtype=float)
        # return np.array(0)
        return np.array([0])
        # pass
        # return state
    
    def step(self, action):
        info = {}
        reward = 0
        done:bool = False
        self.load = action[0]

        print("SB stepping: ", round(action[0], 2))
        print("*"*25)
        print("*"*25)
        print("SB stepping: ", round(action[0], 2), file=fileOut)
        print("*"*25, file=fileOut)
        print("*"*25, file=fileOut)
        

        print("Timestep: ", self.timeStep)
        print("-"*5)
        print("Load: ", round(self.load, 2))
        print("Demand: ", self.demand[self.timeStep])
        print("Timestep: ", self.timeStep, file=fileOut)
        print("-"*5, file=fileOut)
        print("Load: ", round(self.load, 2), file=fileOut)
        print("Demand: ", self.demand[self.timeStep], file=fileOut)


        self.deltaUtilization = abs(self.demand[self.timeStep] - self.load)
        self.penalty = (self.deltaUtilization) ** 2
        
        observation = self.demand[self.timeStep]
        
        print("Penalty: ", round(self.penalty, 2))
        print("Delta ", round(self.deltaUtilization, 2))
        print("Observation: ", observation)
        print("-"*25)
        print("Penalty: ", round(self.penalty, 2), file=fileOut)
        print("Delta ", round(self.deltaUtilization, 2), file=fileOut)
        print("Observation: ", observation, file=fileOut)
        print("-"*25, file=fileOut)
        
        # STILL NEED PEAK DEMAND IN REWARD
        # HOW DO WE CONTRIBUTE TO THIS REWARD?
        reward = 0
        if self.load != 0:
            reward -= self.load + self.penalty
        else:
            reward -= self.load + 1 + self.penalty
            
        # REWARD CAN BE DIFFERENCE BETWEEN COSTS

        self.timeStep += 1
        done = True if self.timeStep > 2 else done
            
        
        return np.array([observation]), reward, done, info
    
    def render(self, mode="human"):
        screen_h = 600
        screen_w = 1000
        if self.viewer is None:
            self.viewer = rendering.Viewer(screen_w, screen_h, "SB")
            
        return self.viewer.render(return_rgb_array=mode == "rgb_array")
    
    def close(self):
        if self.viewer:
            self.viewer.close()
            self.viewer = None

In [8]:
env = SmartBuildingEnv()
#env = DummyVecEnv([lambda: env])
batch_size = 500
ep_per_batch = 10 * batch_size

In [9]:
check_env(env)

SB stepping:  1.08
*************************
*************************
Timestep:  0
-----
Load:  1.08
Demand:  3
Penalty:  3.68
Delta  1.92
Observation:  3
-------------------------
SB stepping:  2.08
*************************
*************************
Timestep:  1
-----
Load:  2.08
Demand:  1
Penalty:  1.16
Delta  1.08
Observation:  1
-------------------------
SB stepping:  1.65
*************************
*************************
Timestep:  2
-----
Load:  1.65
Demand:  1
Penalty:  0.42
Delta  0.65
Observation:  1
-------------------------
SB stepping:  1.18
*************************
*************************
Timestep:  0
-----
Load:  1.18
Demand:  3
Penalty:  3.3
Delta  1.82
Observation:  3
-------------------------
SB stepping:  0.78
*************************
*************************
Timestep:  1
-----
Load:  0.78
Demand:  1
Penalty:  0.05
Delta  0.22
Observation:  1
-------------------------
SB stepping:  1.31
*************************
*************************
Timestep:  2
-----
L

c:\Users\Patrick Wilk\Documents\RL\JJ_first_Look\env\lib\site-packages\stable_baselines3\common\env_checker.py:232: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [10]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=tfLogs)
model.learn(total_timesteps=ep_per_batch, reset_num_timesteps=False)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to /Users/Patrick Wilk/Documents/RL/JJ_first_Look/SURPL-V2-JJ_first_look/JJ_Jupyter/results//logs\PPO_0
SB stepping:  0.55
*************************
*************************
Timestep:  0
-----
Load:  0.55
Demand:  3
Penalty:  5.99
Delta  2.45
Observation:  3
-------------------------
SB stepping:  0.0
*************************
*************************
Timestep:  1
-----
Load:  0.0
Demand:  1
Penalty:  1.0
Delta  1.0
Observation:  1
-------------------------
SB stepping:  0.98
*************************
*************************
Timestep:  2
-----
Load:  0.98
Demand:  1
Penalty:  0.0
Delta  0.02
Observation:  1
-------------------------
SB stepping:  1.29
*************************
*************************
Timestep:  0
-----
Load:  1.29
Demand:  3
Penalty:  2.91
Delta  1.71
Observation:  3
-------------------------
SB stepping:  0.0
*************************
*************************
T

In [11]:
model.save("results/PPO_" + "/BuildingTemp")

c:\Users\Patrick Wilk\Documents\RL\JJ_first_Look\env\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'results\PPO_' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
%tensorboard --logdir {sourceDir + "/logs"}


In [ ]:
fileOut.close()